In [1]:
import sys
print(sys.executable)
print(sys.version)

/Library/Developer/CommandLineTools/usr/bin/python3
3.9.6 (default, Oct 17 2025, 17:15:53) 
[Clang 17.0.0 (clang-1700.4.4.1)]


In [2]:
pip install tensorflow-macos==2.15.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tensorflow-metal==1.1.0

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
## 4 classes
import warnings
from tensorflow.keras.preprocessing.image import ImageDataGenerator

warnings.filterwarnings("ignore")

train_dir = "../week3/alzheimer dataset/train"
test_dir  = "../week3/alzheimer dataset/test"


## object for data augmentation and preprocessing
train_datagen = ImageDataGenerator(
    rescale= 1/.255, ##normalize pixel values
    shear_range = 0.4 , ## slant range, titling medical images might harm its quality
    zoom_range = 0.2 , ## zoom range for scale 
    width_shift_range = 0.2 , ##moving image left to right
    height_shift_range = 0.2, ##moving image top to bottom
    rotation_range = 90 , ## this might harm medical image classification
    horizontal_flip =True,
    fill_mode = 'nearest'
)

test_datagen = ImageDataGenerator(rescale =1/.255)

##load the images ,labels , ensuring grayscale (1 color channel)
train_generator = train_datagen.flow_from_directory(
    train_dir ,
    target_size =(128, 128), ##resize the images 
    color_mode ='grayscale',
    class_mode='categorical',
    
) # default batch size =32

test_generator =test_datagen.flow_from_directory(
    test_dir,
    target_size=(128,128),
    color_mode ='grayscale',
    class_mode='categorical'
)


/Users/sheenachanda/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Found 4649 images belonging to 4 classes.
Found 1813 images belonging to 4 classes.


### Model Building -VGG16

Block - conv1 ,conv2,max pool 

Block * 5 - Flatten - 2 FC layers - softmax output layer

In [5]:
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D , Flatten , Dense , MaxPooling2D ,Dropout,BatchNormalization,GlobalAveragePooling2D
import numpy as np

In [6]:
input_size = (128,128,1)
num_classes=4

model =Sequential()

## Block 1
model.add(Conv2D(filters=64 ,kernel_size =(3,3) , padding ='same' , activation ='relu',
                input_shape= input_size))
model.add(Conv2D(filters =64 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 2
model.add(Conv2D(filters=128 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(Conv2D(filters =128 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 3
model.add(Conv2D(filters=256 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(Conv2D(filters =256 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 4
model.add(Conv2D(filters=512 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(Conv2D(filters =512 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 5
model.add(Conv2D(filters=512 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(Conv2D(filters =512 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Classification Head
model.add(Flatten())
model.add(Dense(units = 4096 , activation ='relu'))
model.add(Dense(units =4096 , activation ='relu'))
model.add(Dense(units= num_classes , activation ='softmax'))

##compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

2026-02-16 12:00:13.304209: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4 Pro
2026-02-16 12:00:13.304240: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 24.00 GB
2026-02-16 12:00:13.304249: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 8.88 GB
2026-02-16 12:00:13.304292: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2026-02-16 12:00:13.304313: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      640       
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 64)        0         
 D)                                                              
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 128)       0         
 g2D)                                                   

In [7]:
model.fit(train_generator , epochs=50)

Epoch 1/50


2026-02-16 12:00:14.195032: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


146/146 [==============================] - 30s 196ms/step - loss: 39.7956 - accuracy: 0.5074
Epoch 2/50
146/146 [==============================] - 29s 196ms/step - loss: 218.0216 - accuracy: 0.4151
Epoch 3/50
146/146 [==============================] - 31s 210ms/step - loss: 6132.5879 - accuracy: 0.4139
Epoch 4/50
146/146 [==============================] - 29s 201ms/step - loss: 546.3751 - accuracy: 0.4913
Epoch 5/50
146/146 [==============================] - 29s 197ms/step - loss: 2106.8840 - accuracy: 0.4829
Epoch 6/50
146/146 [==============================] - 29s 195ms/step - loss: 5.1049 - accuracy: 0.4702
Epoch 7/50
146/146 [==============================] - 28s 193ms/step - loss: 1.8591 - accuracy: 0.5087
Epoch 8/50
146/146 [==============================] - 28s 193ms/step - loss: 1.5388 - accuracy: 0.5216
Epoch 9/50
146/146 [==============================] - 29s 196ms/step - loss: 1.6416 - accuracy: 0.5464
Epoch 10/50
146/146 [==============================] - 29s 197ms/step - l

In [8]:
loss,accuracy_vgg=model.evaluate(test_generator)
print('Test accuracy ', accuracy_vgg)

57/57 [==============================] - 4s 63ms/step - loss: 364718292992.0000 - accuracy: 0.2438
Test accuracy  0.24379481375217438


In [9]:
##model is overfitting  - we can add BN +Dropout + smaller dense layers

In [10]:
input_size = (128,128,1)
num_classes=4

model =Sequential()

## Block 1
model.add(Conv2D(filters=64 ,kernel_size =(3,3) , padding ='same' , activation ='relu',
                input_shape= input_size))
model.add(BatchNormalization())
model.add(Conv2D(filters =64 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 2
model.add(Conv2D(filters=128 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =128 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 3
model.add(Conv2D(filters=256 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =256 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 4
model.add(Conv2D(filters=512 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =512 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 5
model.add(Conv2D(filters=512 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =512 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Classification Head
model.add(Flatten())
model.add(Dense(units = 1024 , activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(units =512 , activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(units= num_classes , activation ='softmax'))

##compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 batch_normalization (Batch  (None, 128, 128, 64)      256       
 Normalization)                                                  
                                                                 
 conv2d_11 (Conv2D)          (None, 128, 128, 64)      36928     
                                                                 
 batch_normalization_1 (Bat  (None, 128, 128, 64)      256       
 chNormalization)                                                
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 64, 64, 64)        0         
 g2D)                                                            
                                                      

In [11]:
###Add globalavg pooling instead of flatten

In [12]:
input_size = (128,128,1)
num_classes=4

model =Sequential()

## Block 1
model.add(Conv2D(filters=64 ,kernel_size =(3,3) , padding ='same' , activation ='relu',
                input_shape= input_size))
model.add(BatchNormalization())
model.add(Conv2D(filters =64 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 2
model.add(Conv2D(filters=128 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =128 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 3
model.add(Conv2D(filters=256 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =256 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 4
model.add(Conv2D(filters=512 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =512 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Block 5
model.add(Conv2D(filters=512 ,kernel_size =(3,3) , padding ='same' , activation ='relu'))
model.add(BatchNormalization())
model.add(Conv2D(filters =512 ,kernel_size =(3,3) , padding ='same',activation ='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size =(2,2) , strides =(2,2)))

##Classification Head
model.add(GlobalAveragePooling2D())
model.add(Dense(units = 1024 , activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(units =512 , activation ='relu'))
model.add(Dropout(0.5))
model.add(Dense(units= num_classes , activation ='softmax'))

##compile the model
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_20 (Conv2D)          (None, 128, 128, 64)      640       
                                                                 
 batch_normalization_10 (Ba  (None, 128, 128, 64)      256       
 tchNormalization)                                               
                                                                 
 conv2d_21 (Conv2D)          (None, 128, 128, 64)      36928     
                                                                 
 batch_normalization_11 (Ba  (None, 128, 128, 64)      256       
 tchNormalization)                                               
                                                                 
 max_pooling2d_10 (MaxPooli  (None, 64, 64, 64)        0         
 ng2D)                                                           
                                                      

In [13]:
##Transfer Learning